<a href="https://colab.research.google.com/github/tioluwaniiyin123/AVA-active-speaker/blob/main/(16%2C2)%20computation%20on%20A100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import timeit
import time
from functools import partial

def matmul(A, B):
    n = len(A)
    C = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]
    return C

# Create A (nxn) random matrix
n = 2
 # you can change this size
A = np.random.rand(n, n)

# Multiply A × A
result = matmul(A, A)




In [32]:
print("Result Matrix C= (A x A):")
print(result)

Result Matrix C= (A x A):
[[0.37694435 0.76468923]
 [0.21844648 0.48101619]]


In [33]:

def get_block(n,p, i, j): #i= bloc krow j = block col
  b= n//p # block_size
  i0, i1 = i *b, (i + 1) * b
  j0, j1 = j * b, (j + 1) * b
  return slice(i0, i1), slice(j0, j1) ## start a index i0(inclusive) stop at index i1(exclusive)



# Simulate one processor computing C_ij block
def matmul_block(A,n, i, j, p):#i,j is matrixes of the block which will result in c
    n = A.shape[0] # access first element
    b = n // p# size of the block
    C_block = np.zeros((b, b))# all zeroes
    for k in range(p):

        i_slice, k_slice = get_block(n, p, i, k)
        k_slice2, j_slice = get_block(n, p, k, j)
        A_ik = A[i_slice, k_slice]
        A_kj = A[k_slice2, j_slice]

         #manual multipliction of the block to get the total block
        for row in range(b):
            for col in range(b):
                total = 0
                for inner in range(b):  # for the inner loop index
                    total += A_ik[row, inner] * A_kj[inner, col]
                C_block[row, col] += total
    return C_block



In [34]:
def parallel_matmul(A, p): #compute_c
    n = A.shape[0]
    b = n // p
    C = np.zeros((n, n))
    #simulate each processor
    for i in range(p):
        for j in range(p):
            # Use i and j as block indices bi and bj
            C_block = matmul_block(A, n, i, j, p)
            i_slice, j_slice = get_block(n, p, i, j)
            C[i_slice, j_slice] = C_block

    return C

In [35]:
import time

def run_test(n, p):
    A = np.random.rand(n, n)

    # Time only the computation
    start = timeit.default_timer()
    C = parallel_matmul(A, p)
    stop = timeit.default_timer()

    execution_time = stop - start

    # Output
    print(f"n = {n}, p = {p}")
    print(f"Block size: {n//p} x {n//p}")
    print(f"Computation time on a A100: {execution_time:.6f} seconds")


In [36]:
run_test(16, 2)

n = 16, p = 2
Block size: 8 x 8
Computation time on a A100: 0.001885 seconds
